# Analyze homes in Pittsburgh

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
CONDITIONDESC_RANKINGS = ['EXCELLENT', 'GOOD', 'AVERAGE', 'FAIR', 'POOR', 'UNSOUND', 'VERY POOR']
DB = create_engine('postgresql://jon@localhost:5432/propertydb')

In [3]:
def pivot_properties(properties):
    propertiespiv = pd.pivot_table(properties, index=['usecode', 'usedesc'], columns='conditiondesc', values='count', aggfunc=sum)

    for ranking in CONDITIONDESC_RANKINGS:
        if ranking not in propertiespiv:
            propertiespiv[ranking] = 0
        propertiespiv[ranking] = propertiespiv[ranking].fillna(0)

    propertiespiv = propertiespiv[CONDITIONDESC_RANKINGS]

    propertiespiv = propertiespiv.reset_index()
    propertiespiv = propertiespiv.set_index('usedesc')
    propertiespiv = propertiespiv.drop('usecode', axis=1)

    # Format capitalization for public display
    propertiespiv.columns = propertiespiv.columns.str.lower()
    propertiespiv.columns = propertiespiv.columns.str.capitalize()
    
    propertiespiv.index = propertiespiv.index.str.lower()
    propertiespiv.index = propertiespiv.index.str.capitalize()

    return propertiespiv

### Residential delinquencies

##### Methodology

##### Run SQL query

In [4]:
properties = pd.read_sql_query("""SELECT usecode, usedesc, conditiondesc, COUNT(DISTINCT assessments.parcelid)
FROM assessments

JOIN parcelliens
ON parcelliens.parcelid = assessments.parcelid

JOIN parceldelinquenciespgh
ON parceldelinquenciespgh.parcelid = assessments.parcelid

JOIN parcelviolationspgh
ON parcelviolationspgh.parcelid = assessments.parcelid

WHERE assessments.municode::int < 200
AND assessments.class = 'R'
AND assessments.usedesc NOT IN ('VACANT LAND', 'RES AUX BUILDING (NO HOUSE)')
AND assessments.changenoticeaddress1 NOT LIKE '%%414%%GRANT%%ST%%'

AND parceldelinquenciespgh.prioryears::int >= 5

AND parcelviolationspgh.desc LIKE '%%VACANT%%'
AND parcelviolationspgh.outcome IN ('CONTINUE TO COURT', 'FOLLOW-UP INVESTIGATION SCHEDULED', 'ISSUE CRIMINAL COMPLAINT', 'REFER TO CLEANUP', 'SEND TO CLEAN & LIEN', 'VIOLATION FOUND')

GROUP BY usecode, usedesc, conditiondesc
ORDER BY usecode ASC;""", DB)

##### Create pivot table by property use, condition

In [5]:
propertiespiv = pivot_properties(properties)
propertiespiv

conditiondesc,Excellent,Good,Average,Fair,Poor,Unsound,Very poor
usedesc,,,,,,,
Single family,0,0,147.0,182.0,48.0,20.0,11.0
Two family,0,0,10.0,11.0,3.0,2.0,1.0
Three family,0,0,1.0,5.0,0.0,0.0,0.0
Four family,0,0,0.0,1.0,0.0,0.0,0.0
Townhouse,0,0,3.0,0.0,0.0,0.0,0.0
Rowhouse,0,0,9.0,24.0,7.0,5.0,1.0
Mobile home,0,0,0.0,0.0,1.0,0.0,0.0


In [6]:
propertiespiv.sum().sum()

492.0

##### Output properties to CSV file

In [7]:
propertiespiv.to_csv('output/residential-delinquencies.csv')

### City-owned residential

##### Run SQL query

In [8]:
properties = pd.read_sql_query("""SELECT usecode, usedesc, conditiondesc, COUNT(DISTINCT assessments.parcelid)
FROM assessments

WHERE assessments.municode::int < 200
AND assessments.class = 'R'
AND assessments.usedesc NOT IN ('VACANT LAND', 'RES AUX BUILDING (NO HOUSE)')
AND assessments.changenoticeaddress1 LIKE '%%414%%GRANT%%ST%%'

GROUP BY usecode, usedesc, conditiondesc
ORDER BY usecode ASC;""", DB)

##### Create pivot table by property use, condition

In [9]:
propertiespiv = pivot_properties(properties)
propertiespiv

conditiondesc,Excellent,Good,Average,Fair,Poor,Unsound,Very poor
usedesc,,,,,,,
Single family,0,1.0,68.0,167.0,66.0,35.0,20.0
Two family,0,0.0,5.0,18.0,7.0,6.0,1.0
Three family,0,0.0,2.0,5.0,3.0,0.0,0.0
Townhouse,0,0.0,2.0,0.0,0.0,0.0,0.0
Rowhouse,0,0.0,13.0,49.0,15.0,16.0,2.0
Condemned/boarded-up,0,0.0,0.0,0.0,0.0,2.0,0.0


In [10]:
propertiespiv.sum().sum()

503.0

##### Output properties to CSV file

In [11]:
propertiespiv.to_csv('output/residential-city.csv')